<a href="https://colab.research.google.com/github/ritwiks9635/My_priject_file_4/blob/main/Named_Entity_Recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Dataset :: https://www.kaggle.com/datasets/debasisdotcom/name-entity-recognition-ner-dataset

In [ ]:
# extract zip file

In [ ]:
from zipfile import ZipFile
data = "/content/https:/www.kaggle.com/datasets/debasisdotcom/name-entity-recognition-ner-dataset/name-entity-recognition-ner-dataset.zip"
with ZipFile(data,"r") as zip:
  zip.extractall()
  print("the data has been extracted ")

the data has been extracted 


In [ ]:
# Machine Learning Named Entity Recognition 
#(NER) is a task of Natural Language
# Processing to identify the named
# entities in a certain
# piece of text.

In [ ]:
# load data using pandas

In [ ]:
import pandas as pd
data = pd.read_csv("/content/NER dataset.csv", encoding = 'unicode_escape')
data.head()

,Sentence #,Word,POS,Tag
0,Sentence: 1,Thousands,NNS,O
1,NaN,of,IN,O
2,NaN,demonstrators,NNS,O
3,NaN,have,VBP,O
4,NaN,marched,VBN,O


Data Preparation

In [ ]:
# we need to do some modifications in
# the data to prepare it in such a
# manner so that it can easily fit
# into a neutral network.

In [ ]:
# from itertools import chain
def get_dict_map(data, token_or_tag):
    tok_2_idx = {}
    idx_2_tok = {}
    
    if token_or_tag == 'token':
        vocab = list(set(data['Word'].to_list()))
    else:
        vocab = list(set(data['Tag'].to_list()))
    
    idx_2_tok = {idx:tok for  idx, tok in enumerate(vocab)}
    tok_2_idx = {tok:idx for  idx, tok in enumerate(vocab)}
    return tok_2_idx, idx_2_tok
token_idx, idx_token = get_dict_map(data, 'token')
tag_idx, idx_tag = get_dict_map(data, 'tag')

In [ ]:
# Now I will transform the columns in
# the data to extract the sequential 
# data

In [ ]:
data["word_idx"] = data["Word"].map(token_idx)
data["tag_idx"] = data["Tag"].map(tag_idx)
data_fillna = data.fillna(method="ffill", axis = 0)
data_new = data_fillna.groupby(['Sentence #'], as_index = False)["Word", "POS", "Tag", "word_idx", "tag_idx"].agg(lambda x: list(x))

<ipython-input-6-b7753cfa6c81>:4: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  data_new = data_fillna.groupby(['Sentence #'], as_index = False)["Word", "POS", "Tag", "word_idx", "tag_idx"].agg(lambda x: list(x))


In [ ]:
data_new.head()

,Sentence #,Word,POS,Tag,word_idx,tag_idx
0,Sentence: 1,"[Thousands, of, demonstrators, have, marched, ...","[NNS, IN, NNS, VBP, VBN, IN, NNP, TO, VB, DT, ...","[O, O, O, O, O, O, B-geo, O, O, O, O, O, B-geo...","[13948, 10456, 14080, 26271, 10709, 22146, 591...","[8, 8, 8, 8, 8, 8, 14, 8, 8, 8, 8, 8, 14, 8, 8..."
1,Sentence: 10,"[Iranian, officials, say, they, expect, to, ge...","[JJ, NNS, VBP, PRP, VBP, TO, VB, NN, TO, JJ, J...","[B-gpe, O, O, O, O, O, O, O, O, O, O, O, O, O,...","[11004, 15302, 33589, 25714, 9479, 30074, 1158...","[7, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, ..."
2,Sentence: 100,"[Helicopter, gunships, Saturday, pounded, mili...","[NN, NNS, NNP, VBD, JJ, NNS, IN, DT, NNP, JJ, ...","[O, O, B-tim, O, O, O, O, O, B-geo, O, O, O, O...","[32887, 11218, 17945, 23374, 29491, 21875, 194...","[8, 8, 2, 8, 8, 8, 8, 8, 14, 8, 8, 8, 8, 8, 1,..."
3,Sentence: 1000,"[They, left, after, a, tense, hour-long, stand...","[PRP, VBD, IN, DT, NN, JJ, NN, IN, NN, NNS, .]","[O, O, O, O, O, O, O, O, O, O, O]","[11779, 9623, 26037, 7289, 24467, 18946, 16247...","[8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8]"
4,Sentence: 10000,"[U.N., relief, coordinator, Jan, Egeland, said...","[NNP, NN, NN, NNP, NNP, VBD, NNP, ,, NNP, ,, J...","[B-geo, O, O, B-per, I-per, O, B-tim, O, B-geo...","[23566, 28040, 14642, 8714, 24375, 33063, 1101...","[14, 8, 8, 12, 6, 8, 2, 8, 14, 8, 7, 8, 7, 8, ..."


Split the dataset

In [ ]:
# Now I will split the data into
# training and test sets. I will create
# a function for splitting the data
# because the LSTM layers accept sequences
# of the same length only.

In [ ]:
from sklearn.model_selection import train_test_split 
from keras_preprocessing.sequence import pad_sequences 
from keras.utils import to_categorical

def get_pad_train_test_val(data_new ,data):
  n_token = len(list(set(data["Word"].to_list())))
  n_tag = len(list(set(data["Tag"].to_list())))
  #Pad tokens
  token = data_new["word_idx"].to_list()
  maxlen = max([len(i) for i in token])
  pad_token = pad_sequences(token, maxlen = maxlen, dtype = "int32", padding = "post", value = n_token-1)
  #Pad Tags and convert it into one hot encoding
  tag = data_group["tag_idx"].to_list()
  pad_tag = pad_sequences(tag, maxlen = maxlen, dtype = "int32", padding = "post", value = tag_idx["O"])
  n_tags = len(tag_idx)
  pad_tags = [to_categorical(i, num_classes = n_tags) for i in pad_tag]

  tokens, test_tokens, tags, test_tags = train_test_split(pad_token, pad_tags, test_size=0.1, random_state = 2020)
  train_tokens, val_tokens, train_tags, val_tags = train_test_split(tokens, tags, test_size = 0.25, random_state = 2023)
  print(
        'train_tokens length:', len(train_tokens),
        '\ntrain_tags length:', len(train_tags),
        '\ntest_tokens length:', len(test_tokens),
        '\ntest_tags:', len(test_tags),
        '\nval_tokens:', len(val_tokens),
        '\nval_tags:', len(val_tags),
    )
  return train_tokens, val_tokens, test_tokens, train_tags, val_tags, test_tags

In [ ]:
train_tokens, val_tokens, test_tokens, train_tags, val_tags, test_tags = get_pad_train_test_val(data_new, data)

train_tokens length: 32372 
train_tags length: 32372 
test_tokens length: 4796 
test_tags: 4796 
val_tokens: 10791 
val_tags: 10791


Training Neural Network

In [ ]:
# 1st import requires model

In [ ]:
import numpy as np
import tensorflow
from tensorflow.keras import Sequential, Model, Input
from tensorflow.keras.layers import LSTM, Embedding, Dense, TimeDistributed, Dropout, Bidirectional
from tensorflow.keras.utils import plot_model
from numpy.random import seed
seed(1)
tensorflow.random.set_seed(2)

In [ ]:
input_dim = len(list(set(data["Word"].to_list())))+1
output_dim = 64
input_len = max([len(i) for i in data_group["word_idx"].to_list()])
n_tags = len(tag_idx)

Create Model

In [ ]:
# Now create helper function for create model architecture

In [ ]:
def get_bilstm_lstm_model():
    model = Sequential()

    # Add Embedding layer
    model.add(Embedding(input_dim=input_dim, output_dim=output_dim, input_length=input_len))

    # Add bidirectional LSTM
    model.add(Bidirectional(LSTM(units=output_dim, return_sequences=True, dropout=0.2, recurrent_dropout=0.2), merge_mode = 'concat'))

    # Add LSTM
    model.add(LSTM(units=output_dim, return_sequences=True, dropout=0.5, recurrent_dropout=0.5))

    # Add timeDistributed Layer
    model.add(TimeDistributed(Dense(n_tags, activation="relu")))

    # Compile model
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    model.summary()
    
    return model

In [ ]:
# Helper function for train our model

In [ ]:
def train_model(X, y, model):
    loss = list()
    for i in range(25):
        # fit model for one epoch on this sequence
        history = model.fit(X, y, batch_size=1000, verbose=1, epochs=1, validation_split=0.2)
        loss.append(history.history['loss'][0])
    return loss

In [ ]:
# Pass dataset in my model with 100 epochs

In [ ]:
results = pd.DataFrame()
model = get_bilstm_lstm_model()
model.summary()
results['with_add_lstm'] = train_model(train_tokens, np.array(train_tags), model)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 104, 64)           2251456   
                                                                 
 bidirectional (Bidirectiona  (None, 104, 128)         66048     
 l)                                                              
                                                                 
 lstm_1 (LSTM)               (None, 104, 64)           49408     
                                                                 
 time_distributed (TimeDistr  (None, 104, 17)          1105      
 ibuted)                                                         
                                                                 
Total params: 2,368,017
Trainable params: 2,368,017
Non-trainable params: 0
_________________________________________________________________
Model: "sequential"
__________________________

Now let’s test our model

In [ ]:
import spacy
from spacy import displacy
nlp = spacy.load('en_core_web_sm')
text = nlp('Hi, My name is Ritwik Sarkar \n I am from India \n I want to work with Google \n My favorite teacher is Andrew NG')
displacy.render(text, style = 'ent', jupyter=True)